In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [ ]:
raw_data = pd.read_csv("./data/dat.csv")

raw_data.info()

In [ ]:
print(raw_data.columns)

In [ ]:
card_data = raw_data.filter(regex='pk|^c').copy()
card_data.dropna()
card_data.head()

In [ ]:
card_data['pk3'] = card_data['pk3'].str.replace('s', 'a')
card_data['pk3'].unique()

In [ ]:
# from sklearn.preprocessing import LabelEncoder

c_col = card_data.select_dtypes(include='object')

# en = LabelEncoder()
ord = OrdinalEncoder()
for col in c_col:
    card_data[col] = ord.fit_transform(card_data[[col]])

card_data.head()

In [ ]:
from collections import defaultdict

m_cols = card_data.filter(regex=r'^c[a-f]\d{2}m$').columns.tolist()
r_cols = card_data.filter(regex=r'^c[a-f]\d{2}r$').columns.tolist()

def extract_type(col_name):
    match = re.match(r'^(c[a-f])\d{2}[mr]$', col_name)
    if match:
        return match.group(1)
    else:
        return None

types_m = [extract_type(col) for col in m_cols]
types_r = [extract_type(col) for col in r_cols]

type_dict = defaultdict(lambda: {'m': [], 'r': []})
for col, t in zip(m_cols, types_m):
    if t:
        type_dict[t]['m'].append(col)
for col, t in zip(r_cols, types_r):
    if t:
        type_dict[t]['r'].append(col)

for t in type_dict:
    type_dict[t]['m'].sort(key=lambda x: int(re.search(r'\d{2}', x).group()))
    type_dict[t]['r'].sort(key=lambda x: int(re.search(r'\d{2}', x).group()))

for t, cols in type_dict.items():
    m_sorted = cols['m']
    r_sorted = cols['r']
    min_len = min(len(m_sorted), len(r_sorted))
    for i in range(min_len):
        m_col = m_sorted[i]
        r_col = r_sorted[i]
        tot_col = f"{m_col[:-1]}_tot"  # Remove 'm' and add '_tot'
        card_data[tot_col] = card_data[m_col] * card_data[r_col]
    if len(m_sorted) > len(r_sorted):
        for m_col in m_sorted[min_len:]:
            tot_col = f"{m_col[:-1]}_tot"
            card_data[tot_col] = np.nan
tot_columns = [col for col in card_data.columns if col.endswith('_tot')]



print("New 'tot' columns created:")
print(tot_columns)


In [ ]:

# Display the first few rows of the new 'tot' columns
print("\nSample of the new 'tot' columns:")
print(card_data[tot_columns].head())

In [ ]:
# card_data['pk3'].value_counts()

card_data_2 = card_data.drop(columns = card_data.filter(regex = r'^c[a-f]\d{2}m$'))

card_data_2 = card_data_2.drop(columns = card_data.filter(regex = r'^c[a-f]\d{2}r$'))

In [ ]:
card_data_2.head()

In [ ]:
card_data_2.dropna()
card_data_2['pk4'] = card_data_2['pk4'].apply(lambda x:x//10000)

In [ ]:
card_data_2['pk5'] = card_data_2['pk4'].apply(lambda x:x//100)
card_data_2['pk4'] = card_data_2['pk4'].apply(lambda x:x%100)

In [ ]:
# Convert pk4, pk5 to categorical
labEn = LabelEncoder()
card_data_2['pk4'] = labEn.fit_transform(card_data_2['pk4'])
card_data_2['pk5'] = labEn.fit_transform(card_data_2['pk5'])

In [ ]:
card_data_other = card_data_2[['pk1', 'pk2', 'pk3', 'pk4', 'pk5', 'cg01r', 'cg02r', 'cg03r', 'cg04r', 'cg05r', 'cg06r', 'cg07r']]

card_data_consume = card_data_2.drop(columns = ['cg01r', 'cg02r', 'cg03r', 'cg04r', 'cg05r', 'cg06r', 'cg07r'])

In [ ]:
card_data_consume = card_data_consume.dropna()
card_data_consume

In [ ]:
card_data_consume.to_csv("./card_data_consume.csv", index = False)

In [ ]:
# Combine into large categories: ca~cf

pattern = re.compile(r'^(c[a-f])\d{2}_tot$')


for char in ['a', 'b', 'c', 'd', 'e','f']:
    name = f"c{char}"
    card_data_consume[name]=card_data_consume[f'c{char}01_tot']

card_data_consume.head()

In [ ]:
card_data_summed = card_data_consume.copy()

for col in card_data_consume.columns:
    if pattern.match(col) and str(col)[2:4] != '01':
        char = str(col)[1]
        card_data_summed[f"c{char}"] += card_data_consume[col]

card_data_summed.head()

In [ ]:
card_data_summed.drop(columns = [col for col in card_data_consume.columns if pattern.match(col)], inplace = True)

In [ ]:
card_data_summed.isnull().sum()

In [ ]:
card_data_other.isnull().sum()

In [ ]:
card_data_summed = card_data_summed.dropna()

In [ ]:
card_data_other = card_data_other.dropna()

In [ ]:
card_data_summed.isnull().sum()

In [ ]:
card_data_other.isnull().sum()

In [ ]:
new_cols = ["기준연월", "성별", "연령대", "시/도", "구/군", "소비", "유통소비", "음식", "생활서비스", "전문서비스", "문화/레저"]
cols = card_data_summed.columns.tolist()
col_dict = dict(zip(cols,new_cols))

card_data_summed.rename(columns = col_dict, inplace=True)

In [ ]:
# Set Korean font in matplotlib
plt.rcParams['font.family'] = 'NanumGothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
# plt.rcParams['font.family'] = 'NanumGothic'  # Linux
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
sns.heatmap(card_data_summed.corr(), annot = True, fmt = ".1f")

In [ ]:
sns.heatmap(card_data_other.corr(), annot = True, fmt = ".1f")

In [ ]:
card_data_summed.to_csv('./card_data_summed.csv', index = False)
card_data_other.to_csv('./card_data_other.csv', index = False)